## Auto ARIMA Pros & Cons

#### Pros
- Saves time
- Removes ambiguity
- Reduce risk of human error

#### Cons
- Blindly putting our faith in one criterion
- Never really see how well the other models perform
- Topic expertise

## Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import yfinance
import warnings
warnings.filterwarnings('ignore')
sns.set()

In [2]:
raw_data = yfinance.download(
    tickers = '^GSPC ^FTSE ^N225 ^GDAXI',
    start = '1994-01-07', end = '2018-01-29',
    interval = '1d', group_by = 'ticker',
    auto_adjust = True, treads = True
)

[*********************100%***********************]  4 of 4 completed


In [3]:
df_comp = raw_data.copy()

In [4]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [5]:
df_comp = df_comp.iloc[1:]
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
del df_comp['^N225']
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method = 'ffill')

## Creating Returns

In [6]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1) * 100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1) * 100
df_comp['ret_dax'] = df_comp.dax.pct_change(1) * 100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1) * 100

## Splitting the Data

In [7]:
size = int(len(df_comp) * 0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

## Fitting a Model

In [8]:
from pmdarima.arima import auto_arima